# Graph learning

<!-- ## JanusGraph-Visualizer queries

- Visualize only the collaboration pattern nodes, situation nodes, and action nodes
  - `g.V().not(hasLabel(containing("participant_").or(eq("robot"))))`
- Visualize the situation vertices of interest
  - `g.V().hasLabel("situation_1", "situation_2")` -->

## Start HumemAI

In [1]:
from humemai.utils import disable_logger
disable_logger()

from gremlin_python.structure.graph import Graph, Vertex, Edge
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.driver.serializer import GraphSONSerializersV3d0
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.traversal import P, T, Direction

import json
from humemai.janusgraph import Humemai
from tqdm.auto import tqdm

humemai = Humemai()
humemai.connect()


# humemai.remove_all_data()
# humemai.disconnect()
# humemai.stop_docker_compose()
# humemai.remove_docker_compose()

from humemai.utils import disable_logger
disable_logger()

/home/tk/.virtualenvs/humemai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
DEBUG:humemai.janusgraph.utils.docker:Starting Docker Compose with project name 'jce' using file '/home/tk/repos/humemai/humemai/janusgraph/docker-compose-cql-es.yml'...
DEBUG:humemai.janusgraph.utils.docker:Waiting for 30 seconds to allow containers to warm up...
INFO:humemai.janusgraph.utils.docker:Docker Compose started successfully.
DEBUG:humemai.janusgraph.utils.docker:
DEBUG:humemai.janusgraph.humemai:Attempting to establish a new connection to the Gremlin server.
DEBUG:humemai.janusgraph.humemai:Gremlin server URL: ws://localhost:8182/gremlin
INFO:gremlinpython:Creating DriverRemoteConnection with url 'ws://localhost:8182/gremlin'
INFO:gremlinpython:Creating Client with url 'ws://localhost:8182/gremlin'
INFO:gremlinpyth

## Write the co-learning data to HumemAI

In [2]:
with open("./raw-data.json") as f:
    data = json.load(f)


humemai.remove_all_data()


for data_point in tqdm(data):
    time_added = data_point["timestamp"]

    robot_vertex = humemai.write_long_term_vertex("robot", {"time_added": time_added})
    cp_properties = {
        "cp_num": data_point["cp_num"],
        "participant_num": data_point["participant"],
        "cp_name": data_point["cp_name"],
        "ticks_lasted": data_point["ticks_lasted"],
        "round_num": data_point["round_num"],
        "time_added": data_point["timestamp"],
        "time_elapsed": data_point["time_elapsed"],
        "remaining_rocks": data_point["remaining_rocks"],
        "victim_harm": data_point["victim_harm"],
    }

    cp_vertex = humemai.write_long_term_vertex("CP", cp_properties)
    humemai.write_long_term_edge(
        robot_vertex, "has_cp", cp_vertex, {"time_added": time_added}
    )

    participant_vertex = humemai.write_long_term_vertex(
        "participant",
        {"participant_number": data_point["participant"], "time_added": time_added},
    )
    humemai.write_long_term_edge(
        participant_vertex, "has_cp", cp_vertex, {"time_added": time_added}
    )

    situation = [bar for foo in data_point["situation"] for bar in foo]

    if situation:
        situation_properties = {s["type"]: s["content"] for s in situation}
        situation_properties["time_added"] = time_added
        situation_vertex = humemai.write_long_term_vertex(
            "situation", situation_properties
        )
        humemai.write_long_term_edge(
            cp_vertex, "has_situation", situation_vertex, {"time_added": time_added}
        )

        for idx, list_ in enumerate(data_point["HumanAction"]):

            if list_:
                properties = {"time_added": time_added}
                for action in list_:
                    properties[action["type"]] = action["content"]
                properties["action_number"] = idx

                human_action_vertex = humemai.write_long_term_vertex(
                    "human_action", properties
                )
                humemai.write_long_term_edge(
                    situation_vertex,
                    "has_human_action_" + str(idx),
                    human_action_vertex,
                    {"time_added": time_added},
                )

        for idx, list_ in enumerate(data_point["RobotAction"]):

            if list_:
                properties = {"time_added": time_added}
                for action in list_:
                    properties[action["type"]] = action["content"]
                properties["action_number"] = idx

                robot_action_vertex = humemai.write_long_term_vertex(
                    "robot_action", properties
                )
                humemai.write_long_term_edge(
                    situation_vertex,
                    "has_robot_action_" + str(idx),
                    robot_action_vertex,
                    {"time_added": time_added},
                )

# # Merge the duplicate nodes
# humemai.connect_duplicate_vertices("exact_label")

100%|██████████| 211/211 [00:02<00:00, 90.90it/s] 
